In [64]:
import requests
import json
import pandas as pd
import numpy as np

graph_id = "QmYkEQD9MFq2tGcB7QX3Znn8H2FNrridwjd9CcKaFVNqJW"
url = f"https://api.thegraph.com/subgraphs/id/{graph_id}"


PASS = "PASS"
FAIL = "FAIL"
MATCH_PERCENT_TOLERANCE = 0.00001 # Much be within 0.001% of eachother

check_summary = {}


In [65]:
# Helpers

def list_items_match(a: list, b: list) -> list:
    exact_match = (a == b)
    within_tolerance = abs(1 - a / b) < MATCH_PERCENT_TOLERANCE
    return exact_match | within_tolerance 
    
def perform_request(query):
    res = requests.post(url, json={'query': query})
    return json.loads(res.text)["data"]


In [66]:
# Check total accounts equals protocol account
query = """query {
    accounts(first: 1000) {
        id
    }
    protocols {
        cumulativeUniqueUsers
    }
}"""

data = perform_request(query)
num_accounts = len(data["accounts"])
cumulative_users = data["protocols"][0]["cumulativeUniqueUsers"]

result = PASS if num_accounts == cumulative_users else FAIL 

print(f"num_accounts: {num_accounts}, cumulative_users: {cumulative_users}, result: {result}")

check_summary["unique_user_check"] = result


num_accounts: 43, cumulative_users: 43, result: PASS


In [67]:
# Check cumulative for protocol match sum for pools

query = """query {
    lendingProtocols {
        totalValueLockedUSD
        cumulativeSupplySideRevenueUSD
        cumulativeProtocolSideRevenueUSD
        cumulativeTotalRevenueUSD
        totalDepositBalanceUSD
        cumulativeDepositUSD
        totalBorrowBalanceUSD
        cumulativeBorrowUSD
        cumulativeLiquidateUSD
    }
    markets {
        totalValueLockedUSD
        _cumulativeSupplySideRevenueUSD
        _cumulativeProtocolSideRevenueUSD
        _cumulativeTotalRevenueUSD
        totalDepositBalanceUSD
        cumulativeDepositUSD
        totalBorrowBalanceUSD
        cumulativeBorrowUSD
        cumulativeLiquidateUSD
    }
}"""

data = perform_request(query)
protcol_data = data["lendingProtocols"][0]
market_data = data["markets"]
df = pd.DataFrame(market_data)
df.rename(columns = {'_cumulativeSupplySideRevenueUSD':'cumulativeSupplySideRevenueUSD', '_cumulativeProtocolSideRevenueUSD':'cumulativeProtocolSideRevenueUSD', "_cumulativeTotalRevenueUSD": "cumulativeTotalRevenueUSD"}, inplace = True)
df = df.apply(pd.to_numeric, errors='coerce') 
market_sums = df.sum(axis=0)

compare_df = pd.DataFrame({"market_sums": market_sums, "protocol_cumulatives": protcol_data})
compare_df = compare_df.apply(pd.to_numeric, errors='coerce') 
compare_df["result"] = np.where(list_items_match(compare_df["market_sums"], compare_df["protocol_cumulatives"]), PASS, FAIL)
print(compare_df)

result = PASS if (compare_df["result"].to_numpy() == PASS).all() else FAIL

check_summary["protocol_cumulatives_match_check"] = result



                                   market_sums  protocol_cumulatives result
totalValueLockedUSD               1.750403e+07          1.750403e+07   PASS
cumulativeSupplySideRevenueUSD    0.000000e+00          0.000000e+00   PASS
cumulativeProtocolSideRevenueUSD  0.000000e+00          0.000000e+00   PASS
cumulativeTotalRevenueUSD         0.000000e+00          0.000000e+00   PASS
totalDepositBalanceUSD            1.700000e+07          1.700000e+07   PASS
cumulativeDepositUSD              1.700000e+07          1.700000e+07   PASS
totalBorrowBalanceUSD             1.300000e+07          1.300000e+07   PASS
cumulativeBorrowUSD               1.300000e+07          1.300000e+07   PASS
cumulativeLiquidateUSD            0.000000e+00          0.000000e+00   PASS


In [68]:
# Check financial snapshots add up to protocol_cumulatives

query = """query {
    lendingProtocols {
        cumulativeSupplySideRevenueUSD
        cumulativeProtocolSideRevenueUSD
        cumulativeDepositUSD
        cumulativeBorrowUSD
        cumulativeLiquidateUSD
    }
    financialsDailySnapshots {
        dailySupplySideRevenueUSD
        dailyProtocolSideRevenueUSD
        dailyDepositUSD
        dailyBorrowUSD
        dailyLiquidateUSD
    }
}"""


data = perform_request(query)
protcol_data = data["lendingProtocols"][0]
snapshot_data = data["financialsDailySnapshots"]
df = pd.DataFrame(snapshot_data)
df = df.apply(pd.to_numeric, errors='coerce') 
df.rename(columns = {'dailySupplySideRevenueUSD':'cumulativeSupplySideRevenueUSD', 'dailyProtocolSideRevenueUSD':'cumulativeProtocolSideRevenueUSD', "dailyDepositUSD": "cumulativeDepositUSD", "dailyBorrowUSD": "cumulativeBorrowUSD", "dailyLiquidateUSD": "cumulativeLiquidateUSD"}, inplace = True)
sums = df.sum(axis=0)

compare_df = pd.DataFrame({"snapshot_sums": sums, "protocol_cumulatives": protcol_data})
compare_df = compare_df.apply(pd.to_numeric, errors='coerce') 
compare_df["result"] = np.where(list_items_match(compare_df["snapshot_sums"], compare_df["protocol_cumulatives"]), PASS, FAIL)
print(compare_df)

result = PASS if (compare_df["result"].to_numpy() == PASS).all() else FAIL

check_summary["financial_snapshots_check"] = result


                                  snapshot_sums  protocol_cumulatives result
cumulativeSupplySideRevenueUSD     0.000000e+00          0.000000e+00   PASS
cumulativeProtocolSideRevenueUSD   0.000000e+00          0.000000e+00   PASS
cumulativeDepositUSD               1.700000e+07          1.700000e+07   PASS
cumulativeBorrowUSD                1.300000e+07          1.300000e+07   PASS
cumulativeLiquidateUSD             0.000000e+00          0.000000e+00   PASS


In [69]:
# Check market snapshots add up to market cumulatives

query = """query {
    markets(first: 1) {
        cumulativeDepositUSD
        cumulativeBorrowUSD
        cumulativeLiquidateUSD

        hourlySnapshots {
            hourlyDepositUSD
            hourlyBorrowUSD
            hourlyLiquidateUSD
        } 

        dailySnapshots {
            dailyDepositUSD
            dailyBorrowUSD
            dailyLiquidateUSD
        }
    }
   
}"""

data = perform_request(query) 
market_data = data["markets"][0]
filtered_market_data = {"cumulativeDepositUSD": float(market_data["cumulativeDepositUSD"]), "cumulativeBorrowUSD": float(market_data["cumulativeBorrowUSD"]), "cumulativeLiquidateUSD": float(market_data["cumulativeLiquidateUSD"])}

hourly_snapshot_data = market_data["hourlySnapshots"]
hourly_snapshot_df = pd.DataFrame(hourly_snapshot_data)
hourly_snapshot_df = hourly_snapshot_df.apply(pd.to_numeric, errors='coerce') 
hourly_snapshot_df.rename(columns = {'hourlyDepositUSD':'cumulativeDepositUSD', 'hourlyBorrowUSD':'cumulativeBorrowUSD', "hourlyLiquidateUSD": "cumulativeLiquidateUSD"}, inplace = True)
hourly_snapshot_sums = hourly_snapshot_df.sum(axis=0) 

daily_snapshot_data = market_data["dailySnapshots"]
daily_snapshot_df = pd.DataFrame(daily_snapshot_data)
daily_snapshot_df = daily_snapshot_df.apply(pd.to_numeric, errors='coerce') 
daily_snapshot_df.rename(columns = {'dailyDepositUSD':'cumulativeDepositUSD', 'dailyBorrowUSD':'cumulativeBorrowUSD', "dailyLiquidateUSD": "cumulativeLiquidateUSD"}, inplace = True)
daily_snapshot_sums = daily_snapshot_df.sum(axis=0) 

compare_df = pd.DataFrame({"market_data": filtered_market_data, "hourly_snapshot_sums": hourly_snapshot_sums, "daily_snapshot_sums": daily_snapshot_sums})
compare_df["result"] = np.where(list_items_match(compare_df["market_data"], compare_df["hourly_snapshot_sums"]) & list_items_match(compare_df["market_data"], compare_df["daily_snapshot_sums"]), PASS, FAIL)
print(compare_df)

result = PASS if (compare_df["result"].to_numpy() == PASS).all() else FAIL
check_summary["market_snapshots_check"] = result



                         market_data  hourly_snapshot_sums  \
cumulativeDepositUSD    1.700000e+07          1.700000e+07   
cumulativeBorrowUSD     1.500000e+07          1.500000e+07   
cumulativeLiquidateUSD  0.000000e+00          0.000000e+00   

                        daily_snapshot_sums result  
cumulativeDepositUSD           1.700000e+07   PASS  
cumulativeBorrowUSD            1.500000e+07   PASS  
cumulativeLiquidateUSD         0.000000e+00   PASS  


In [70]:
# Summarize results

print(check_summary)

{'unique_user_check': 'PASS', 'protocol_cumulatives_match_check': 'PASS', 'financial_snapshots_check': 'PASS', 'market_snapshots_check': 'PASS'}
